In [1]:
from selenium import webdriver
import time
import requests
import pandas as pd
import openpyxl
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm_notebook

from datetime import date
from datetime import timedelta

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

In [13]:
start_time = time.time()

imdb_id_list = []
title_list = []
year_list = []
time_list = []
overview_list = []
director_list = []
starring_list = []
url_list = []
genre_list = []
studio_list = []

imdb_id_list_2 = []
title_list_2 = []
year_list_2 = []
time_list_2 = []
overview_list_2 = []
director_list_2 = []
starring_list_2 = []
url_list_2 = []
genre_list_2 = []
studio_list_2 = []

url = 'https://www.imdb.com/list/ls029217360/'

resp = requests.get(url)
soup = BeautifulSoup(resp.text)

# 날짜 범위에 따라 긁어오는 범위 지정하기

month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

temp_all = []
year_list_temp = []

links = soup.select('h3', class_ = 'lister-item-header')

for i in range(1, len(links) - 3):
    dates = soup.select('#main > div > div.lister.list.detail.sub-list > div.lister-list > div:nth-child(' + str(i) + ') > div.list-description > p')
    
    for j in dates:
        date_text = j.get_text().strip()
        date_loc_1 = date_text.find('-')
        date_loc_2 = date_text.find('Studio')
        month_text = date_text[date_loc_1 + 2:date_loc_2 - 1]
        
        movie_release_date = month_text.split()
        
        
        if movie_release_date[0] != 'TBA':
            for k in range(len(month_list)):
                if movie_release_date[0] == month_list[k]:
                    temp_month = (k + 1)
        
            temp_day = int(movie_release_date[1][:-2])
            temp_year = int(movie_release_date[2])
        else:
            temp_month = 12
            temp_day = 31
            temp_year = int(movie_release_date[1])
        
        temp_all.append(date(temp_year, temp_month, temp_day))
        
        year_list_temp.append(temp_year)

        
for i in range(len(temp_all)):
    if temp_all[i] > date.today() - timedelta(days = 60):
        new_movie_date_index = i
        break
        
for i in range(len(temp_all)):
    if temp_all[i] > date.today() + timedelta(days = 7):
        upcoming_movie_date_index = i
        break
        
print('New movie starting point : ', new_movie_date_index + 1, ' Upcoming movie starting point : ', upcoming_movie_date_index + 1)





# 최신 영화(기준 : 현재일로부터 2달전~ 1주일후) 크롤링하기------------------------------------------------

print('Recent Movies Selenium Crawling Start...')

# 영화 줄거리, 감독, 출연 배우 긁어오기

path_ = '../../chromedriver/chromedriver'

url = 'https://www.imdb.com/list/ls029217360/'

driver = webdriver.Chrome(path_)

driver.get(url)

time.sleep(3)

soup = BeautifulSoup(driver.page_source)


for i in range(new_movie_date_index, upcoming_movie_date_index):
    urls = driver.find_element_by_xpath('//*[@id="main"]/div/div[4]/div[3]/div[' + str(i + 1) + ']/div[2]/h3/a')
    site_url = urls.get_attribute('href')
    url_list.append(site_url)
    
    url_loc_1 = site_url.find('title')
    url_loc_2 = site_url.find('?ref_')
    imdb_id_list.append(site_url[url_loc_1 + 6:url_loc_2 - 1])
    
    overviews = driver.find_element_by_xpath('//*[@id="main"]/div/div[4]/div[3]/div[' + str(i + 1) + ']/div[2]/p[2]')
    overview_list.append(overviews.text)
    
    director_starring = driver.find_element_by_xpath('//*[@id="main"]/div/div[4]/div[3]/div[' + str(i + 1) + ']/div[2]/p[3]')
    
    if director_starring.text != '':
        director_loc_1 = director_starring.text.find(':')
        director_loc_2 = director_starring.text.find('|')
        starring_loc = director_starring.text.find('Stars')
        if starring_loc == -1:
            starring_loc_exception = director_starring.text.find('Star')
            
        director_list.append(director_starring.text[director_loc_1 + 2:director_loc_2 - 1])
        
        if starring_loc != -1:
            starring_list.append(director_starring.text[starring_loc + 7:])
        else:
            starring_list.append(director_starring.text[starring_loc_exception + 6:])
    else:
        director_list.append('None')
        starring_list.append('None')
        
    time.sleep(1)

driver.close()

print('Recent Movies Selenium Crawling Ended...')

print('Recent Movies BeautifulSoup Crawling Start...')

# 영화 제목, 개봉년도 긁어오기

for i in range(new_movie_date_index, upcoming_movie_date_index):
    val = links[i]
    title_year = val.get_text().strip()
    
    title_loc = title_year.find('.')
    title_exception_loc = title_year.find('(I')
    year_loc = title_year.find('(20')
    
    if title_exception_loc == -1:
        if year_loc != -1:
            title_list.append(title_year[title_loc + 2:year_loc - 1])
        else:
            title_list.append(title_year[title_loc + 2:])
    else:
        title_list.append(title_year[title_loc+2:title_exception_loc-1])
    
    year_list.append(year_list_temp[i])
        
        
# 영화 상영 시간 긁어오기

for i in range(new_movie_date_index, upcoming_movie_date_index):
    runtimes = soup.select('#main > div > div.lister.list.detail.sub-list > div.lister-list > div:nth-child(' + str(i + 1) + ') > div.lister-item-content > p:nth-child(2) > span.runtime')
    
    if runtimes:
        for j in runtimes:
            time_pre = j.get_text().strip()
            time_loc = time_pre.find('min')
            time_list.append(time_pre[:time_loc-1])
    else:
        time_list.append('None')

# 영화 장르 긁어오기

for i in range(new_movie_date_index, upcoming_movie_date_index):
    genres = soup.select('#main > div > div.lister.list.detail.sub-list > div.lister-list > div:nth-child(' + str(i + 1) + ') > div.lister-item-content > p:nth-child(2) > span.genre')
    
    if genres:
        for j in genres:
            genre_list.append(j.get_text().strip())
    else:
        genre_list.append('None')

# 영화 제작사 긁어오기

for i in range(new_movie_date_index, upcoming_movie_date_index):
    dates = soup.select('#main > div > div.lister.list.detail.sub-list > div.lister-list > div:nth-child(' + str(i + 1) + ') > div.list-description > p')
    
    for j in dates:
        date_text = j.get_text().strip()
        studio_loc = date_text.find('Studio')
        
        if date_text[studio_loc + 9:] != '':
            studio_list.append(date_text[studio_loc + 9 :])
        else:
            studio_list.append('None')
            
print('Recent Movies Crawling Ended...')
            
            
        
        
        
        
            
# 개봉 예정 영화(기준 : 현재일로부터 1주일후) 크롤링하기------------------------------------------------

print('Upcoming Movies Selenium Crawling Start...')

# 영화 줄거리, 감독, 출연 배우 긁어오기

path_ = '../../chromedriver/chromedriver'

url = 'https://www.imdb.com/list/ls029217360/'

driver = webdriver.Chrome(path_)

driver.get(url)

time.sleep(3)

soup = BeautifulSoup(driver.page_source)


for i in range(upcoming_movie_date_index, len(links) - 4):
    urls = driver.find_element_by_xpath('//*[@id="main"]/div/div[4]/div[3]/div[' + str(i + 1) + ']/div[2]/h3/a')
    site_url = urls.get_attribute('href')
    url_list_2.append(site_url)
    
    url_loc_1 = site_url.find('title')
    url_loc_2 = site_url.find('?ref_')
    imdb_id_list_2.append(site_url[url_loc_1 + 6:url_loc_2 - 1])
    
    overviews = driver.find_element_by_xpath('//*[@id="main"]/div/div[4]/div[3]/div[' + str(i + 1) + ']/div[2]/p[2]')
    overview_list_2.append(overviews.text)
    
    director_starring = driver.find_element_by_xpath('//*[@id="main"]/div/div[4]/div[3]/div[' + str(i + 1) + ']/div[2]/p[3]')
    
    if director_starring.text != '':
        director_loc_1 = director_starring.text.find(':')
        director_loc_2 = director_starring.text.find('|')
        starring_loc = director_starring.text.find('Stars')
        if starring_loc == -1:
            starring_loc_exception = director_starring.text.find('Star')
            
        director_list_2.append(director_starring.text[director_loc_1 + 2:director_loc_2 - 1])
        
        if starring_loc != -1:
            starring_list_2.append(director_starring.text[starring_loc + 7:])
        else:
            starring_list_2.append(director_starring.text[starring_loc_exception + 6:])
    else:
        director_list_2.append('None')
        starring_list_2.append('None')
        
    time.sleep(1)
    
driver.close()

print('Upcoming Movies Selenium Crawling Ended...')

print('Upcoming Movies BeautifulSoup Crawling Start...')


# 영화 제목, 개봉년도 긁어오기

for i in range(upcoming_movie_date_index, len(links) - 4):
    val = links[i]
    title_year = val.get_text().strip()
    
    title_loc = title_year.find('.')
    title_exception_loc = title_year.find('(I')
    year_loc = title_year.find('(20')
    
    if title_exception_loc == -1:
        if year_loc != -1:
            title_list_2.append(title_year[title_loc + 2:year_loc - 1])
        else:
            title_list_2.append(title_year[title_loc + 2:])
    else:
        title_list_2.append(title_year[title_loc+2:title_exception_loc-1])
        
    year_list_2.append(year_list_temp[i])
        
        
# 영화 상영 시간 긁어오기

for i in range(upcoming_movie_date_index, len(links) - 4):
    runtimes = soup.select('#main > div > div.lister.list.detail.sub-list > div.lister-list > div:nth-child(' + str(i + 1) + ') > div.lister-item-content > p:nth-child(2) > span.runtime')
    
    if runtimes:
        for j in runtimes:
            time_pre = j.get_text().strip()
            time_loc = time_pre.find('min')
            time_list_2.append(time_pre[:time_loc-1])
    else:
        time_list_2.append('None')

# 영화 장르 긁어오기

for i in range(upcoming_movie_date_index, len(links) - 4):
    genres = soup.select('#main > div > div.lister.list.detail.sub-list > div.lister-list > div:nth-child(' + str(i + 1) + ') > div.lister-item-content > p:nth-child(2) > span.genre')
    
    if genres:
        for j in genres:
            genre_list_2.append(j.get_text().strip())
    else:
        genre_list_2.append('None')

# 영화 제작사 긁어오기

for i in range(upcoming_movie_date_index, len(links) - 4):
    dates = soup.select('#main > div > div.lister.list.detail.sub-list > div.lister-list > div:nth-child(' + str(i + 1) + ') > div.list-description > p')
    
    for j in dates:
        date_text = j.get_text().strip()
        studio_loc = date_text.find('Studio')
        
        if date_text[studio_loc + 9:] != '':
            studio_list_2.append(date_text[studio_loc + 9 :])
        else:
            studio_list_2.append('None')
            
print('Upcoming Movies Crawling Ended...') 
            
            
            
            
            
items_new = pd.DataFrame({'imdb_id' : imdb_id_list, 'title' : title_list, 'release' : year_list, 
                      'genres' : genre_list, 'director' : director_list, 'actor' : starring_list,
                      'country' : np.nan, 'runtime' : time_list, 'production' : studio_list, 'overview' : overview_list,
                      'url' : url_list})

items_upcoming = pd.DataFrame({'imdb_id' : imdb_id_list_2, 'title' : title_list_2, 'release' : year_list_2, 
                      'genres' : genre_list_2, 'director' : director_list_2, 'actor' : starring_list_2,
                      'country' : np.nan, 'runtime' : time_list_2, 'production' : studio_list_2, 'overview' : overview_list_2,
                      'url' : url_list_2})



items_new.to_excel('IMDB_Crawling_Recent.xlsx')
items_upcoming.to_excel('IMDB_Crawling_Upcoming.xlsx')



print('Spend time : ', time.time() - start_time)

New movie starting point :  13  Upcoming movie starting point :  33
Recent Movies Selenium Crawling Start...
Recent Movies Selenium Crawling Ended...
Recent Movies BeautifulSoup Crawling Start...
Recent Movies Crawling Ended...
Upcoming Movies Selenium Crawling Start...
Upcoming Movies Selenium Crawling Ended...
Upcoming Movies BeautifulSoup Crawling Start...
Upcoming Movies Crawling Ended...
Spend time :  179.19364094734192
